# NLP Code Along

For this code along we will build a spam filter! We'll use the various NLP tools we learned about as well as a new classifier, Naive Bayes.

We'll use a classic dataset for this - UCI Repository SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [41]:
from pyspark.sql import SparkSession

In [42]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [43]:
data = spark.read.csv("/content/drive/MyDrive/Spark with Python/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Natural_Language_Processing/smsspamcollection",inferSchema=True,sep='\t')

In [44]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [45]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



## Clean and Prepare the Data

** Create a new length feature: **

In [46]:
from pyspark.sql.functions import length

In [47]:
data = data.withColumn('length',length(data['text']))

In [48]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [49]:
# Pretty Clear Difference
data.groupby('class').mean().show()

+--------------------+-----------------+
|               class|      avg(length)|
+--------------------+-----------------+
|                 ham|71.45431945307645|
|                spam|138.6706827309237|
|We would apprecia...|             null|
|spam   URGENT! Yo...|             null|
|1. Tiago Agostinh...|             null|
|3. Limitation of ...|             null|
|ham   Cos i was o...|             null|
|    1.1. Compilation|             null|
|      1. DESCRIPTION|             null|
|The SMS Spam Coll...|             null|
|Note: messages ar...|             null|
|We offer a compre...|             null|
|-----------------...|             null|
|         1.3. Format|             null|
|            --------|             null|
|- A collection of...|             null|
|The files contain...|             null|
|The corpus has be...|             null|
|2. No Warranty/Us...|             null|
|ham   Siva is in ...|             null|
+--------------------+-----------------+
only showing top

## Feature Transformations

In [50]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer

tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [51]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [52]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

### The Model

We'll use Naive Bayes, but feel free to play around with this choice!

In [53]:
from pyspark.ml.classification import NaiveBayes

In [54]:
# Use defaults
nb = NaiveBayes()

### Pipeline

In [55]:
from pyspark.ml import Pipeline

In [56]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [57]:
cleaner = data_prep_pipe.fit(data)

Py4JJavaError: ignored

In [ ]:
clean_data = cleaner.transform(data)

### Training and Evaluation!

In [ ]:
clean_data = clean_data.select(['label','features'])

In [ ]:
clean_data.show()

In [ ]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [ ]:
spam_predictor = nb.fit(training)

In [ ]:
data.printSchema()

In [ ]:
test_results = spam_predictor.transform(testing)

In [ ]:
test_results.show()

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Not bad considering we're using straight math on text data! Try switching out the classification models! Or even try to come up with other engineered features!

## Great Job!